In [4]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import math

In [5]:
dataset = pd.read_csv("usnews_dataset.csv")
print("Raw data")
# Display first 5 rows
dataset.head(5)

Raw data


,Name,Ranking,usnews.com link,Tuition and Fees,Room and Board,Total Enrollment,School Type,Year Founded,Religiious Affiliation,Academic Calendar,...,Total undergraduate enrollment,Undergraduates who are first generation,Out-of-state students,International students,Registered clubs and organizations,Number of sports,Unnamed: 76,Peer_assessment,First_year_top_ten,Social Mobility rank
0,Williams College,1,https://www.usnews.com/best-colleges/williams-...,"$57,280",14990,2127.0,"Private, Coed",1793.0,None,04-01-04,...,2073.0,22%,86%,7.80%,154.0,17.0,NaN,4.7,89%,90.0
1,Amherst College,2,https://www.usnews.com/best-colleges/amherst-c...,"$58,640",15310,1855.0,"Private, Coed",1821.0,None,Semester,...,1855.0,21%,87%,8.10%,177.0,12.0,NaN,4.6,88%,99.0
2,Swarthmore College,3,https://www.usnews.com/best-colleges/swarthmor...,"$54,656",16088,1559.0,"Private, Coed",1864.0,None,Semester,...,1559.0,27%,87%,13.20%,154.0,8.0,NaN,4.6,90%,148.0
3,Wellesley College,3,https://www.usnews.com/best-colleges/wellesley...,"$56,052",17096,2534.0,"Private, Women's college",1870.0,None,Semester,...,2534.0,17%,86%,13.60%,153.0,8.0,NaN,4.5,83%,113.0
4,Pomona College,5,https://www.usnews.com/best-colleges/pomona-co...,"$54,762",17218,1679.0,"Private, Coed",1887.0,None,Semester,...,1679.0,30%,74%,11.40%,227.0,9.0,NaN,4.5,91%,113.0


**Filter the columns of interest + drop rows that contains null values (it drops from 216 down to 168)**
- Name
- Ranking 
- Peer_assessment 
- Total Enrollment 
- 2018 Endowment
- Student_faculty_ratio
- Freshmen_retention
- First_year_top_ten


In [6]:
def filter_dataset(dataset):
    usnews = dataset[['Name', \
                      'Ranking', \
                      'Peer_assessment', \
                      'Total Enrollment', \
                      '2018 Endowment', \
                      'Student_faculty_ratio', \
                      'Freshmen_retention', \
                      'First_year_top_ten',
                      'Acceptance_rate']]
    # Drop rows which contains N/A values
    return usnews.dropna()

**Data cleaning and engineering**

In [7]:
# Format 3 rows: endowment, student-faculty ratio and students from top 10 in high school
def endowment_format(s):
    s = s.replace(',','').split('.')[0]
    return float(s[1:])
def ratio_format(s):
    return int(s.split(":")[0])
def percentage_format(s):
    return int(s[:-1])/100

def clean_data(usnews):
    usnews["2018 Endowment"] = usnews["2018 Endowment"].apply(endowment_format)
    usnews["Student_faculty_ratio"] = usnews["Student_faculty_ratio"].apply(ratio_format)
    usnews["First_year_top_ten"] = usnews["First_year_top_ten"].apply(percentage_format)
    usnews["Freshmen_retention"] = usnews["Freshmen_retention"].apply(percentage_format)
    usnews["Acceptance_rate"] = usnews["Acceptance_rate"].apply(percentage_format)
    
    return usnews

def new_variables(usnews):
    usnews["endowment_per_capita"] = usnews["2018 Endowment"]/usnews['Total Enrollment']
    usnews["log_endowment_per_capita"] = np.log(usnews["endowment_per_capita"])
    return usnews

**Regression**

In [8]:
def regressions():
    ranking = ols(formula = 'Ranking ~ log_endowment_per_capita + \
                                       Student_faculty_ratio + \
                                       First_year_top_ten + \
                                       Acceptance_rate + \
                                       Freshmen_retention', data = usnews).fit()
    peer = ols(formula = 'Peer_assessment ~ log_endowment_per_capita + \
                                            Student_faculty_ratio + \
                                            First_year_top_ten + \
                                            Acceptance_rate + \
                                            Freshmen_retention', data = usnews).fit()
    
    ranking_endowment = ols(formula = 'Ranking ~ log_endowment_per_capita', data = usnews).fit()
    peer_endowment = ols(formula = 'Peer_assessment ~ log_endowment_per_capita', data = usnews).fit()
    return ranking, peer, ranking_endowment, peer_endowment

**Predict**

In [9]:
def predict(regressions):
    ranking = regressions[0]
    peer = regressions[1]
    ranking_endowment = regressions[2]
    peer_endowment = regressions[3]
    
    predict_ranking = []
    predict_peer_score = []
    predict_ranking_endowment = []
    predict_peer_endowment = []

    for i in range(len(usnews)):
        predict_ranking.append(round(
                               ranking.params['Intercept']+ \
                               ranking.params['log_endowment_per_capita']*usnews['log_endowment_per_capita'].values[i] + \
                               ranking.params['Student_faculty_ratio']*usnews['Student_faculty_ratio'].values[i] + \
                               ranking.params['First_year_top_ten']*usnews['First_year_top_ten'].values[i] + \
                               ranking.params['Freshmen_retention']*usnews['Freshmen_retention'].values[i] \
                               ))
        predict_peer_score.append(round(
                                  peer.params['Intercept']+ \
                                  peer.params['log_endowment_per_capita']*usnews['log_endowment_per_capita'].values[i] + \
                                  peer.params['Student_faculty_ratio']*usnews['Student_faculty_ratio'].values[i] + \
                                  peer.params['First_year_top_ten']*usnews['First_year_top_ten'].values[i] + \
                                  peer.params['Freshmen_retention']*usnews['Freshmen_retention'].values[i] \
                                  ,2))
        predict_ranking_endowment.append(round(
                                         ranking_endowment.params['Intercept'] + \
                                         ranking_endowment.params['log_endowment_per_capita']*usnews['log_endowment_per_capita'].values[i]))
        predict_peer_endowment.append(round(
                                      peer_endowment.params['Intercept'] + \
                                      peer_endowment.params['log_endowment_per_capita']*usnews['log_endowment_per_capita'].values[i],2))
    return predict_ranking, predict_peer_score, predict_ranking_endowment, predict_peer_endowment

**Save the comparasion between actual data and predicted data to .csv file**

In [10]:
def save(usnews, predict_ranking, predict_peer_score, predict_ranking_endowment, predict_peer_endowment):
    (pd.DataFrame({"Name": usnews["Name"], \
                   "Actual Ranking":usnews['Ranking'], \
                   "Model predicted ranking":predict_ranking, \
                   "Model predicted ranking by log_endowment_per_capita":predict_ranking_endowment, \
                   "Actual Peer Assessment":usnews["Peer_assessment"], \
                   "Model predicted peer assessment":predict_peer_score, \
                   "Model predicted peer_assessment by log_endowment_per_capita":predict_peer_endowment, \
                   })).to_csv("model_predicted_modified.csv")

**Pipeline**

In [14]:
usnews = filter_dataset(dataset)
usnews = clean_data(usnews)
usnews = new_variables(usnews)
ranking_regression, peer_regression, ranking_endowment_regression, peer_endowment_regression = regressions()

**Save model**

In [15]:
predict_ranking, predict_peer_score, predict_ranking_endowment, predict_peer_endowment = \
                predict([ranking_regression, peer_regression, ranking_endowment_regression, peer_endowment_regression])

# save(usnews, predict_ranking, predict_peer_score, predict_ranking_endowment, predict_peer_endowment)
print("Saved")

Saved


**Descriptive statistics**

In [ ]:
usnews.describe()

In [ ]:
ranking_regression, peer_regression, ranking_endowment_regression, peer_endowment_regression = regressions()
print('Ranking vs. log endowment result')
ranking_endowment_regression.summary()

In [ ]:
print('Peer assessment vs. log endowment result')
peer_endowment_regression.summary()

In [16]:
print('Ranking vs. other factors result')
ranking_regression.summary()

Ranking vs. other factors result


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Ranking   R-squared:                       0.861
Model:                            OLS   Adj. R-squared:                  0.857
Method:                 Least Squares   F-statistic:                     201.1
Date:                Tue, 23 Jun 2020   Prob (F-statistic):           1.50e-67
Time:                        15:23:01   Log-Likelihood:                -752.39
No. Observations:                 168   AIC:                             1517.
Df Residuals:                     162   BIC:                             1536.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                  379.3777     34.847     10.887      0.000     310.564     448.191
log_endowment_per_capita    -8.1229      1.818     -4.469      0.000     -11.713      -4.533
Student_faculty_ratio        2.9572      1.134      2.607      0.010       0.718       5.197
First_year_top_ten         -34.9256     13.836     -2.524      0.013     -62.247      -7.604
Acceptance_rate             38.8100     11.550      3.360      0.001      16.002      61.618
Freshmen_retention        -279.7671     25.795    -10.846      0.000    -330.706    -228.829
==============================================================================
Omnibus:                        8.932   Durbin-Watson:                   1.713
Prob(Omnibus):                  0.011   Jarque-Bera (JB):               10.064
Skew:                           0.410   Prob(JB):                      0.00653
Kurtosis:                       3.874   Cond. No.                         361.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
print('Peer assessment vs. other factors result')
peer_regression.summary()

**-------------------------------------------------------------**

**Table**

In [84]:
college = ["Williams College", "Pomona College", "Claremont McKenna College", "Middlebury College", "Bates College", \
             "Bard College", "Berea College", "Skidmore College", "Linfield College", "Bennington College", \
             "Haveford College"]

In [85]:
usnews.head()

,Name,Ranking,Peer_assessment,Total Enrollment,2018 Endowment,Student_faculty_ratio,Freshmen_retention,First_year_top_ten,Acceptance_rate,endowment_per_capita,log_endowment_per_capita
0,Williams College,1,4.7,2127.0,2.600000e+09,7,0.98,0.89,0.13,1.222379e+06,14.016309
1,Amherst College,2,4.6,1855.0,2.400000e+09,7,0.97,0.88,0.13,1.293801e+06,14.073095
2,Swarthmore College,3,4.6,1559.0,2.100000e+09,8,0.98,0.90,0.09,1.347017e+06,14.113403
3,Wellesley College,3,4.5,2534.0,2.100000e+09,8,0.96,0.83,0.20,8.287293e+05,13.627649
4,Pomona College,5,4.5,1679.0,2.300000e+09,8,0.97,0.91,0.08,1.369863e+06,14.130221


In [86]:
# Create new dataframe with colleges of interest only
table = pd.DataFrame()
for i in college:
    table = table.append(usnews[usnews['Name'] == i], ignore_index = True)

In [87]:
table

,Name,Ranking,Peer_assessment,Total Enrollment,2018 Endowment,Student_faculty_ratio,Freshmen_retention,First_year_top_ten,Acceptance_rate,endowment_per_capita,log_endowment_per_capita
0,Williams College,1,4.7,2127.0,2.600000e+09,7,0.98,0.89,0.13,1.222379e+06,14.016309
1,Pomona College,5,4.5,1679.0,2.300000e+09,8,0.97,0.91,0.08,1.369863e+06,14.130221
2,Claremont McKenna College,7,4.3,1327.0,8.353000e+08,8,0.95,0.78,0.09,6.294650e+05,13.352625
3,Middlebury College,7,4.3,2626.0,1.100000e+09,8,0.95,0.78,0.17,4.188880e+05,12.945359
4,Bates College,21,4.1,1832.0,3.156000e+08,10,0.95,0.55,0.18,1.722707e+05,12.056823
5,Bard College,62,3.5,2218.0,1.520000e+08,9,0.85,0.41,0.65,6.853021e+04,11.135030
6,Berea College,46,3.5,1673.0,1.200000e+09,10,0.83,0.22,0.38,7.172744e+05,13.483214
7,Skidmore College,39,3.6,2613.0,3.793000e+08,8,0.92,0.38,0.27,1.451588e+05,11.885584
8,Linfield College,117,2.7,1376.0,1.245000e+08,9,0.82,0.30,0.81,9.047965e+04,11.412880
9,Bennington College,89,2.9,811.0,3.500000e+07,9,0.78,0.55,0.57,4.315660e+04,10.672591


In [88]:
# A dictionary for new columns

avg_ranking, avg_peer_assessment, avg_endowment_per_cap, avg_stu_fac_ratio, avg_retention, avg_first_year_top_10, \
avg_acceptance_rate = [],[],[],[],[],[],[]

avg = { "avg_ranking" : ["Ranking", avg_ranking],
        "avg_peer_assessment" : ["Peer_assessment", avg_peer_assessment],
        "avg_endowment_per_cap" : ["endowment_per_capita", avg_endowment_per_cap],
        "avg_stu_fac_ratio" : ["Student_faculty_ratio", avg_stu_fac_ratio],
        "avg_retention" : ["Freshmen_retention", avg_retention],
        "avg_first_year_top_10" : ["First_year_top_ten", avg_first_year_top_10],
        "avg_acceptance_rate" : ["Acceptance_rate", avg_acceptance_rate]}


In [89]:
# update array of columns
for i in avg:
    for j in college:
        upper = math.ceil(usnews[usnews["Name"] == j]["Ranking"].tolist()[0]/10)*10
        lower = upper - 9
        if lower < 100: # ranking group < 100
            avg[i][1].append(round(usnews[(usnews["Ranking"] >= lower) & (usnews["Ranking"] <= upper)][avg[i][0]].mean(),2))
        else: # ranking group > 100
            avg[i][1].append(round(usnews[(usnews["Ranking"] >= 101)][avg[i][0]].mean(),2))
# update new columns for average to the dataframe
for i in avg:
    table[i] = avg[i][1]            

In [90]:
table.to_csv('Table.csv')
print('Saved')

Saved
